<a href="https://colab.research.google.com/github/KangTuna/Machine-learning-and-deep-learning-practices/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets

In [ ]:
# 디바이스 할당
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
image_sz = 64
fake_sz = 100
lat_dimension = 64

In [ ]:
transform = transforms.Compose([transforms.Resize((image_sz,image_sz)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,)),
])
data_root = '/content/drive/MyDrive/Colab Notebooks/report_file'

In [ ]:
# train_dataset = datasets.MNIST(
#     root="./data", train=True, transform=transform, download=True ,)

train_dataset = datasets.ImageFolder(root = data_root, transform=transform)

In [ ]:
batch_size = 128

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
train_loader

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
       # 생성자를 구성하는 층 정의
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(fake_sz, 512, kernel_size=4),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )
    def forward(self, x):
        return self.gen(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
       # 판별자를 구성하는 층의 정의
        self.disc = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 3, kernel_size=4),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.disc(x)

In [ ]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)
# print(generator)
# print(discriminator)

In [ ]:
optim_g = optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5,0.999))
optim_d = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5,0.999))

In [ ]:
criterion = nn.BCELoss()

losses_g = []
losses_d = []

In [ ]:
from torchvision.utils import save_image

In [ ]:
for idx, data in enumerate(train_loader):
  print('idx')
  print(idx)
  print('data')
  print(data)
  image,_ = data
  print('image')
  print(image)
  b_size = image.shape # image.shape로 바꿔서 한번 해봐
  print('b_size')
  print(b_size)
  break


In [ ]:
c_size = image.shape[:2]
print(c_size)

torch.Size([128, 3])


In [ ]:
epochs = 1000
for epoch in range(epochs):
    loss_g = 0.0
    loss_d = 0.0
    for idx, data in enumerate(train_loader):
        optim_d.zero_grad()
        image, _ = data
        data_real = image.to(device)
        b_size = data_real.shape[:2]
        real_label = torch.ones(b_size).to(device)
        fake_label = torch.zeros(b_size).to(device)

        output_real = discriminator(data_real)
        loss_real = criterion(torch.squeeze(output_real), real_label)

        noise = torch.randn(b_size[0], fake_sz, 1, 1).to(device) # 노이즈 확인하셈
        data_fake = generator(noise)
        output_fake = discriminator(data_fake.detach())
        loss_fake = criterion(torch.squeeze(output_fake), fake_label)

        loss_total = (loss_real + loss_fake)/2
        loss_total.backward()
        optim_d.step()

        optim_g.zero_grad()
        output = discriminator(data_fake)
        g_loss = criterion(torch.squeeze(output), real_label)
        g_loss.backward()
        optim_g.step()

        loss_g += g_loss
        loss_d += loss_total

    epoch_loss_g = loss_g / idx
    epoch_loss_d = loss_d / idx
    losses_g.append(epoch_loss_g)
    losses_d.append(epoch_loss_d)
    print(f"Epoch {epoch} of {epochs}")
    print(f"Generator loss: {epoch_loss_g:.8f}, Discriminator loss: {epoch_loss_d:.8f}")
    save_image(data_fake[:25],
              f'{epoch}.png',
              nrow=5,
              normalize=True)